# Parsing a Counter-Strike 2 Demo

In this notebook, we show how to install Awpy and parse a Counter-Strike 2 demo file, also called a replay file or simply, demo. To start, install Awpy via pip by running `pip install awpy`. Python >= 3.11 is a prerequisite! You can get a demo either through the game itself, or by visiting a website like [HLTV](https://www.hltv.org/) or [FACEIT](https://www.faceit.com/). Then, to parse the demo, you can run the following:

In [1]:
from awpy import Demo

# Demo: https://www.hltv.org/matches/2372746/spirit-vs-natus-vincere-blast-premier-spring-final-2024 (de_dust2, Map 2)
dem = Demo("spirit-vs-natus-vincere-m2-dust2.dem", verbose=True)
dem.parse()

# Available properties (all demos)
print(f"\nHeader: \n{dem.header}")
print(f"\nRounds: \n{dem.rounds.head(n=3)}")
print(f"\nKills: \n{dem.kills.head(n=3)}")
print(f"\nDamages: \n{dem.damages.head(n=3)}")
print(f"\nWeapon Fires: \n{dem.shots.head(n=3)}")
print(f"\nBomb: \n{dem.bomb.head(n=3)}")
print(f"\nSmokes: \n{dem.smokes.head(n=3)}")
print(f"\nInfernos: \n{dem.infernos.head(n=3)}")
print(f"\nGrenades: \n{dem.grenades.head(n=3)}")
print(f"\nFootsteps: \n{dem.footsteps.head(n=3)}")
print(f"\nTicks: \n{dem.ticks.head(n=3)}")



2025-02-17 11:27:34.708 | DEBUG    | awpy.demo:parse:214 - Starting to parse spirit-vs-natus-vincere-m2-dust2.dem
2025-02-17 11:27:39.170 | SUCCESS  | awpy.demo:parse:258 - Finished parsing spirit-vs-natus-vincere-m2-dust2.dem, took 4.46 seconds

Header: 
{'game_directory': '/home/csserver001/cs2/game/csgo', 'allow_clientside_entities': True, 'demo_version_guid': '8e9d71ab-04a1-4c01-bb61-acfede27c046', 'server_name': 'BLAST Premier 2024', 'demo_file_stamp': 'PBDEMS2\x00', 'client_name': 'SourceTV Demo', 'allow_clientside_particles': True, 'fullpackets_version': '2', 'addons': '', 'demo_version_name': 'valve_demo_2', 'map_name': 'de_dust2', 'network_protocol': '14011'}

Rounds: 
shape: (3, 9)
┌───────────┬───────┬────────────┬───────┬───┬────────┬──────────┬────────────┬─────────────┐
│ round_num ┆ start ┆ freeze_end ┆ end   ┆ … ┆ winner ┆ reason   ┆ bomb_plant ┆ bomb_site   │
│ ---       ┆ ---   ┆ ---        ┆ ---   ┆   ┆ ---    ┆ ---      ┆ ---        ┆ ---         │
│ u32       ┆ i32

### Getting player and global properties
Awpy uses [demoparser2](https://github.com/LaihoE/demoparser) as its parsing backend. This means that you can pass a list of `player_props` or `other_props`. In the following example, we get some player position properties. If you do not pass any props, we choose a default list of properties (which is already quite extensive). To see a list of available properties, visit demoparser2's repository.

In [3]:
dem = Demo("spirit-vs-natus-vincere-m2-dust2.dem")
dem.parse(player_props=["X", "Y", "Z", "health", "armor_value", "has_helmet", "has_defuser", "inventory"])

print(f"\nTicks: \n{dem.ticks.head(n=3)}")

2025-02-17 11:27:58.733 | DEBUG    | awpy.demo:parse:214 - Starting to parse spirit-vs-natus-vincere-m2-dust2.dem
2025-02-17 11:28:09.006 | SUCCESS  | awpy.demo:parse:258 - Finished parsing spirit-vs-natus-vincere-m2-dust2.dem, took 10.27 seconds

Ticks: 
shape: (3, 12)
┌─────────────┬────────┬─────────────┬────────────┬───┬──────┬─────────────┬───────────┬───────────┐
│ inventory   ┆ health ┆ has_defuser ┆ has_helmet ┆ … ┆ tick ┆ steamid     ┆ name      ┆ round_num │
│ ---         ┆ ---    ┆ ---         ┆ ---        ┆   ┆ ---  ┆ ---         ┆ ---       ┆ ---       │
│ list[str]   ┆ i32    ┆ bool        ┆ bool       ┆   ┆ i32  ┆ u64         ┆ str       ┆ u32       │
╞═════════════╪════════╪═════════════╪════════════╪═══╪══════╪═════════════╪═══════════╪═══════════╡
│ ["knife_but ┆ 100    ┆ false       ┆ false      ┆ … ┆ 577  ┆ 76561198386 ┆ donk      ┆ 1         │
│ terfly",    ┆        ┆             ┆            ┆   ┆      ┆ 265483      ┆           ┆           │
│ "USP-S"]    ┆       

### Obtaining all events
Because we use the demoparser2 backend, we can parse many kinds of events in the demo. To see the default list, you can check `Demo.default_events`. These events are parsed unless specified otherwise (via the `events=[...]` argument in `.parse()`). To access parsed events, after parsing a demo, simply check the `.events` property, which is a dictionary where the key is the event name and the value is the parsed Polars dataframe.

In [5]:
for event_name, event in dem.events.items():
    print(f"{event_name}: {event.shape[0]} rows x {event.shape[1]} columns")

bomb_planted: 12 rows x 13 columns
player_given_c4: 24 rows x 12 columns
bomb_dropped: 63 rows x 13 columns
round_freeze_end: 23 rows x 2 columns
bomb_defused: 2 rows x 13 columns
player_spawn: 242 rows x 12 columns
inferno_startburn: 108 rows x 16 columns
smokegrenade_expired: 135 rows x 16 columns
player_death: 165 rows x 53 columns
weapon_fire: 3402 rows x 14 columns
bomb_pickup: 75 rows x 12 columns
player_sound: 23783 rows x 15 columns
inferno_expire: 108 rows x 16 columns
bomb_exploded: 6 rows x 13 columns
smokegrenade_detonate: 146 rows x 16 columns
round_end: 22 rows x 5 columns
item_pickup: 1724 rows x 15 columns
player_hurt: 588 rows x 29 columns
round_start: 24 rows x 3 columns
hegrenade_detonate: 64 rows x 16 columns
round_officially_ended: 44 rows x 2 columns
flashbang_detonate: 183 rows x 16 columns
